In [43]:
import parser

In [ ]:
# def get_idents(paper):
#     xid = dict()
#     for x in paper.identifiers:
#         xid[x.key_type] = x.key
#     if paper.title:
#         xid['title'] = paper.title
#     return xid

# def compare_idents(pap1, pap2):
#     for key, value in pap1.items():
#         if pap2.get(key, False) == value:
#             return True
#     return False

In [126]:
import gzip, msgpack



def get_doi(paper):
    _ref_dict = dict()
    for x in paper.identifiers:
        _ref_dict[x.key_type] = x.key
    if 'doi' in _ref_dict:
        return _ref_dict['doi']
    elif 'ieee' in _ref_dict:
        return _ref_dict['ieee']
    elif 'semanticscholar' in _ref_dict:
        return _ref_dict['semanticscholar']
    elif 'adsa' in _ref_dict:
        return _ref_dict['adsa']
    else:
        return False
#     _ref_doi = [x.key for x in paper.identifiers if x.key_type=='doi']
#     if len(_ref_doi):
#         return _ref_doi[0]
#     else:
#         return False

identies_set = dict()

paper_list = dict()
myiter = 0
count=0
ref_dict = dict()
training_set = list()
test_set = list()
with gzip.open("NN_Papers.msgpack.gz", "rb") as nn_papers_out:
    unpacker = msgpack.Unpacker(nn_papers_out, encoding='utf-8')
    for _paper in unpacker:
        count+=1
        for iden in paper.identifiers:
            if identies_set.get(iden.key_type, False):
                identies_set[iden.key_type]+=1
            else:
                identies_set[iden.key_type]=1
        paper = parser.Paper.deserialize(_paper)
        
        paper_doi = get_doi(paper)
        if not (paper_doi and paper.abstract):
            continue
        if myiter >=20000: #and ref_dict.get(paper_doi, False):
            break
        else:
            myiter+=1
        
#         if myiter >=20000: #and ref_dict.get(paper_doi, False):
#             continue
#         else:
#             myiter+=1
        # _paper is a python dict object. you can use it directly if you don't want to use the Paper class provided here

        ref_dict[paper_doi] = list()
        for ref in paper.references:
            ref_doi = get_doi(ref)
            if not ref_doi:
                continue
            
            if ref_dict.get(ref_doi, False):
                ref_dict[ref_doi].append(paper_doi)                
            else:
                ref_dict[ref_doi] = [paper_doi]

            
            ref_dict[paper_doi].append(ref_doi)
            
        
        paper_list[paper_doi] = paper
        if myiter<=16000:
            training_set.append(paper)
        else: 
            test_set.append(paper)

print('total count: {0}'.format(count))
print(identies_set)

total count: 28811
{'nature': 241, 'semanticscholar': 5603, 'ieee': 6969, 'arxiv': 1016, 'pmcid': 1, 'oxford': 718, 'onepetro': 130, 'sage': 86, 'mid': 45, 'science': 10, 'doi': 22034, 'pmc': 792, 'adsa': 4926, 'iop': 368, 'springer': 1731, 'pmid': 3080, 'manuscript': 87, 'acm': 5386}


In [127]:
from sklearn.feature_extraction.text import HashingVectorizer
N_FEATURES = 2**18
hv = HashingVectorizer(n_features=N_FEATURES,non_negative=True, stop_words='english')

In [128]:
abstract_training_set = [paper.abstract for paper in training_set]
abstract_training_set_hashed = hv.transform(abstract_training_set)

In [129]:
from torch.nn.modules import PairwiseDistance

In [130]:
import random
random.randint(0,100)

90

In [131]:
identies_set = dict()
for paper in paper_list.values():
    for iden in paper.identifiers:
        if identies_set.get(iden.key_type, False):
            identies_set[iden.key_type]+=1
        else:
            identies_set[iden.key_type]=1
print(identies_set)
len(paper_list)

{'springer': 1674, 'semanticscholar': 5369, 'ieee': 6924, 'science': 10, 'arxiv': 1016, 'pmcid': 1, 'oxford': 455, 'onepetro': 85, 'sage': 86, 'mid': 43, 'nature': 233, 'doi': 15280, 'pmc': 746, 'adsa': 4542, 'iop': 367, 'acm': 3823, 'pmid': 2607, 'manuscript': 87}


20000

In [132]:
import random


def get_triple(val_set):
    count = 0
    for item in val_set:
        count += 1
        print('count: {0}'.format(count))
        _ref_doi = get_doi(item)
        if _ref_doi not in ref_dict.keys():
            continue
        _ref_list = ref_dict[_ref_doi]
        print(_ref_list)
        for _ref in _ref_list:
            
            if paper_list.get(_ref, False) in val_set:
                
                _neg_example = None
                
                while(not _neg_example):
                    _neg_id = random.randint(0,len(val_set))
                    _neg_doi = get_doi(val_set[_neg_id])
                    if (_neg_doi != _ref_doi) and (_neg_doi not in _ref_list):
                        _neg_example = val_set[_neg_id]
                
#                 count+=1
                yield (item, paper_list[_ref], _neg_example)

In [202]:
iterat = get_triple(training_set)
a1,a2,a3 = next(iterat)

count: 1
['10.1111/j.1467-8640.1988.tb00284.x', '10.1109/AAMAS.2004.129', '10.1007/978-3-540-30200-1_12', '10.1016/j.eswa.2011.01.045', '10.1109/IAT.2005.125', '10.1109/TNN.2007.905839', '10.1007/978-3-540-72383-7_128', '10.1007/BF00992698', '10.1016/S0734-189X(87)80014-2', '10.1364/ao.26.004919', '10.1109/TSMCB.2007.907040', '10.1007/978-3-642-38786-9_17', '10.1177/105971230000800302', '10.1080/0951508042000239048', '10.1016/0364-0213(90)90002-e', '10.1037/0096-3445.120.3.235', '10.1007/11553939_5', '7f60261da4f7e69951eb15bb4b8b8973a86e27e9', '10.1109/IAT.2006.100', '10.1109/ijcnn.2002.1007545', '10.1109/TNN.2004.826220', '10.1016/0893-6080(94)00092-Z', '10.1016/0893-6080(91)90056-B', '10.1007/bf00198465', '10.1109/WIIAT.2008.29']


In [166]:
# Code in file tensor/two_layer_net_tensor.py
import torch.nn as nn

In [244]:
import torch.nn.functional as F

class ParallelTable(nn.Module):

    def __init__(self):
        super(ParallelTable, self).__init__()
        n1 = nn.Linear(5, 2)
        n2 = nn.Linear(5, 2)
        n1.weight = n2.weight
        n1.bias = n2.bias
        
        self.n1 = n1
        self.n2 = n2

    def forward(self, x):
        a = self.n1(x[0])
        b = self.n2(x[1])
        return (a,b)

    
class DistRes(nn.Module):

    def __init__(self):
        super(DistRes, self).__init__()
        self.layer1 = nn.Linear(20, 2)
        self.layer2 = nn.PairwiseDistance(2)

    def forward(self, x,y):
        res1 = self.layer1(x)
        res2 = self.layer1(y)
        return self.layer2(res1,res2)

In [245]:
prl = ParallelTable()
mlp= nn.Sequential(prl, nn.PairwiseDistance(2))
mlp2 = DistRes()

In [250]:
from torch.autograd import Variable
import torch

arg1 = Variable(torch.from_numpy(hv.transform([a1.abstract]).todense()))
arg2 = Variable(torch.from_numpy(hv.transform([a2.abstract]).todense()))

arg1 = Variable(torch.rand(1,5))
arg2 = Variable(torch.rand(1,5))

arg1

Variable containing:
 0.4227  0.5567  0.8317  0.9628  0.5208
[torch.FloatTensor of size 1x5]

In [251]:
prl([arg1,arg2])[0]

Variable containing:
-0.6993  0.1771
[torch.FloatTensor of size 1x2]

In [ ]:
input = autograd.Variable(hv.transform([a1.abstract]))

In [154]:
import numpy as np

k1 = hv.transform([a1.abstract])
k2 = hv.transform([training_set[0].abstract])
# for k in k2:
#     print(k)
np.sum(k1-k2)==0.0
# (hv.transform([a1.abstract])==hv.transform([training_set[0].abstract])).all()

True

In [ ]:
def generator_batch(iterable, n=100):
    l = len(iterable)
    for ndx in range(0,l,n):
        yield iterable[ndx: min(ndx+n,l)]
        
# paper_list_full = list(paper_list.values())

In [ ]:
abstract_training_set = [paper.abstract for paper in training_set]

In [ ]:
abstract_training_set_hashed = hv.transform(abstract_training_set)

In [6]:
# Network Parameters
n_hidden_1 = 2**12 # 1st layer number of features
n_hidden_2 = 2**8 # 2nd layer number of features
n_input = N_FEATURES # MNIST data input (img shape: 28*28)
n_classes = 2**6 # MNIST total classes (0-9 digits)
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
# Create model
def multilayer_perceptron(x, weights, biases):
# Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
# Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
# Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
return out_layer

# Store layers weight & bias
weights = {
'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
'b1': tf.Variable(tf.random_normal([n_hidden_1])),
'b2': tf.Variable(tf.random_normal([n_hidden_2])),
'out': tf.Variable(tf.random_normal([n_classes]))
}
# Construct model
pred = multilayer_perceptron(x, weights, biases)

[[101 102 103]
 [105 106 107]]


In [ ]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
# Initializing the variables
init = tf.global_variables_initializer()
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
# Training cycle
for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
# Loop over all batches
for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
# Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
# Compute average loss
            avg_cost += c / total_batch
# Display logs per epoch step
if epoch % display_step == 0:
print("Epoch:", '%04d' % (epoch+1), "cost=", \
"{:.9f}".format(avg_cost))
print("Optimization Finished!")
# Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))